# **Malaria Classification**



In [ ]:
!git clone https://github.com/deepanrajm/machine_learning.git

In [2]:
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imutils import paths
import cv2
import numpy as np
import pickle as cPickle
from google.colab.patches import cv2_imshow

In [3]:
def LBPH(image, eps=1e-7):

		lbp = feature.local_binary_pattern(image, numPoints,
			radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, numPoints + 3),
			range=(0, numPoints + 2))

		hist = hist.astype("float")
		hist /= (hist.sum() + eps)

		return hist

In [4]:
numPoints = 24
radius = 8

data = []
labels = []

In [ ]:
for imagePath in paths.list_images("/content/machine_learning/Classification/Maleria_Classification"):
	image = cv2.imread(imagePath)
	print (imagePath)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	hist = LBPH(gray)
	labels.append(imagePath.split("/")[-2])
	data.append(hist)

In [11]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data),
	np.array(labels), test_size=0.25, random_state=42)

In [12]:
model = RandomForestClassifier(n_estimators=20, random_state=42)
model.fit(trainData, trainLabels)
f = open("classifier.cPickle", "wb")
f.write(cPickle.dumps(model))
f.close()

In [ ]:
predictions = model.predict(testData)

print(classification_report(testLabels, predictions))

In [7]:
loaded_model = cPickle.load(open("classifier.cPickle", 'rb'))

In [ ]:
imagePath = "/content/machine_learning/Classification/Maleria_Classification/Uninfected/C3thin_original_IMG_20150608_162835_cell_144.png"
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hist = LBPH(gray)
prediction = loaded_model.predict(hist.reshape(1, -1))[0]
cv2.putText(image, prediction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 255), 1)
cv2_imshow(image)